In [1]:
import os
os.environ["HTTP_PROXY"] = "http://172.17.101.14:8080"
os.environ["HTTPS_PROXY"] = "http://172.17.101.14:8080"
os.environ["NO_PROXY"] = "localhost,127.0.0.1"
os.environ["WANDB_BASE_URL"] = "https://api.wandb.ai"

import wandb
wandb.login(key="b01a12c039f0eb2134dd777020e02cc772beda1d")
wandb.init(project="mistral-ALPACA-finetuning")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\AIC MUJ W2\_netrc
wandb: Currently logged in as: deepdey1226 (deepdey1226-manipal-university-jaipur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
import os
os.environ["HTTP_PROXY"] = "http://172.17.101.14:8080"
os.environ["HTTPS_PROXY"] = "http://172.17.101.14:8080"
os.environ["NO_PROXY"] = "localhost,127.0.0.1"
from huggingface_hub import notebook_login
notebook_login()

In [9]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

In [14]:
# Load and preprocess the dataset
data = load_dataset("tatsu-lab/alpaca", split="train")
data_df = data.to_pandas()
data_df = data_df[:5000]
data_df["text"] = data_df[["input", "instruction", "output"]].apply(lambda x: "###Human: " + x["instruction"] + " " + x["input"] + " ###Assistant: "+ x["output"], axis=1)
data = Dataset.from_pandas(data_df)

In [15]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token

In [17]:
# Configure 4-bit quantization using bitsandbytes
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [18]:
# Load the original Mistral 7B model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=quantization_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
# Prepare the model for training
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [20]:
# Configure LoRA
peft_config = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)

In [21]:
# Set up training arguments
training_arguments = TrainingArguments(
    output_dir="mistral-finetuned-alpaca",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=20,
    num_train_epochs=1,
    max_steps=500,
    fp16=True,
    push_to_hub=True,
    report_to="wandb"
)

In [22]:
# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    #dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    #packing=False,
    #max_seq_length=512
)

C:\Users\AIC MUJ W2\AppData\Local\Temp\ipykernel_14852\2343866527.py:2: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/5000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [23]:
# Start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
c:\Users\AIC MUJ W2\Deep-Iot-C-LLm\ev1\Lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
20,1.570000
40,1.257400
60,1.217200
80,1.180100
100,1.194800
120,1.161200
140,1.215400
160,1.145700
180,1.195500
200,1.217200


TrainOutput(global_step=500, training_loss=1.2024276847839355, metrics={'train_runtime': 956.2236, 'train_samples_per_second': 4.183, 'train_steps_per_second': 0.523, 'total_flos': 3.265759838502912e+16, 'train_loss': 1.2024276847839355})

In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Path to the specific checkpoint directory
checkpoint_dir = "C:/Users/AIC MUJ W2/Deep-Iot-C-LLm/Mistral-Chatbot-v3/mistral-finetuned-alpaca/checkpoint-500"

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",  # Original base model
    device_map="auto",  # Automatically maps the model to available devices (e.g., GPU)
    torch_dtype=torch.float16  # Use FP16 for faster inference (optional)
)

# Load the fine-tuned adapter weights from the checkpoint
model = PeftModel.from_pretrained(base_model, checkpoint_dir)

# Load the tokenizer from the checkpoint
tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)

# Prepare the input prompt
input_text = """###Human: Why mobile is bad for human? ###Assistant: """
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

# Generate a response
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        max_new_tokens=200,  # Adjust the number of tokens to generate
        temperature=0.7,  # Controls randomness (lower = more deterministic)
        top_p=0.9,  # Nucleus sampling: limits sampling to the top p probability mass
        do_sample=True,  # Enable sampling for more diverse outputs
    )

# Decode the generated tokens to text
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


###Human: Why mobile is bad for human? ###Assistant: 1. Mobile devices can be distracting and can lead to a loss of productivity.
2. Mobile devices can be addictive and can cause people to become dependent on them.
3. Mobile devices can be dangerous and can cause people to become careless with their safety.
4. Mobile devices can be expensive and can cause people to become financially stressed.
5. Mobile devices can be a source of stress and can cause people to become overwhelmed.
6. Mobile devices can be a source of cyberbullying and can cause people to become vulnerable.
7. Mobile devices can be a source of addiction and can cause people to become dependent on them.
8. Mobile devices can be a source of privacy issues and can cause people to become vulnerable to hackers.
9. Mobile devices can be a source of distraction and can cause people to become less focused.
10. Mobile devices can be a source of strain on relationships and can cause people to become
